## Import the libraries 

In [268]:
import pandas as pd
import numpy as np
import pandas as pd
import fuzzy_pandas as fpd

In [270]:
di = {"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado","CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington","WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}


In [267]:
#since the data is too large for Github, I uploaded the dataset on google drive 
#https://drive.google.com/file/d/1L9WlxgJmudO8iM-n2OCFUpdPBjBlLUhi/view?usp=sharing
df = pd.read_csv("/Users/asmaaaly/Minerva/SF_spring/SS154/SS154-assignment/Data/Most-Recent-Cohorts-All-Data-Elements.csv")
df2 = pd.read_csv("https://raw.githubusercontent.com/asmaaalaa99/SS154-Final-Project/main/Data/population_data.csv", header=0)


In [269]:
df['STABBR'] = df['STABBR'].map(di)
df.rename({'CITY': 'City','STABBR':'State'}, axis=1, inplace=True)
df['Geographic Area'] = df[['City', 'State']].apply(lambda x: ' , '.join(x[x.notnull()]), axis = 1)
df2['Geographic Area'] = df2["Geographic Area"].apply(lambda x: pd.Series(str(x).replace("city","")))
df2["Geographic Area"] = df2["Geographic Area"].str.lower()
df["Geographic Area"] = df["Geographic Area"].str.lower()
df2 = df2[['2019','Geographic Area']]


In [230]:
matches = fpd.fuzzy_merge(df, df2, on = ['Geographic Area'],
                          ignore_case=True,
                    method = "levenshtein", threshold = 0.9)


        UNITID    OPEID  OPEID6                                INSTNM  \
0       100663   105200    1052   University of Alabama at Birmingham   
1       100690  2503400   25034                    Amridge University   
2       100706   105500    1055   University of Alabama in Huntsville   
3       100724   100500    1005              Alabama State University   
4       100751   105100    1051             The University of Alabama   
...        ...      ...     ...                                   ...   
3366  48511103   157101    1571   Georgia Military College - Valdosta   
3367  48511106   157105    1571     Georgia Military College - Robins   
3368  48511107   157107    1571   Georgia Military College - Columbus   
3369  49005401  4182601   41826  HCI College - Fort Lauderdale Campus   
3370  49146401  4227001   42270      ABC Adult School - Cabrillo Lane   

                 City       State         ZIP  \
0          Birmingham     Alabama  35294-0110   
1          Montgomery    

In [240]:
columns_of_interest= ["MD_EARN_WNE_P10", "MD_EARN_WNE_P8", "MD_EARN_WNE_P6", "INSTNM", "2019", "GRAD_DEBT_MDN","FIRST_GEN","MARRIED","ADM_RATE","SAT_AVG","POVERTY_RATE","Geographic Area","LN_MEDIAN_HH_INC","COSTT4_A","UGDS_WHITE","MEDIAN_HH_INC","MD_EARN_WNE_P8","MD_EARN_WNE_P10"]
regression_columns = ["MD_EARN_WNE_P10", "MD_EARN_WNE_P8", "MD_EARN_WNE_P6", "INSTNM", "population_2019", "GRAD_DEBT_MDN","FIRST_GEN","MARRIED","POVERTY_RATE","Geographic Area","LN_MEDIAN_HH_INC","COSTT4_A","UGDS_WHITE","MEDIAN_HH_INC","MD_EARN_WNE_P10"]

In [241]:
raw_data = matches.copy()[columns_of_interest]
raw_data.rename({'2019': 'population_2019'}, axis=1, inplace=True)
raw_data = raw_data.replace('PrivacySuppressed', np.nan)
clean_data = raw_data.dropna(subset=regression_columns)
clean_data["population_2019"] = clean_data["population_2019"].str.replace(',', '').astype(float)
clean_data.to_csv("clean_data.csv")